In [ ]:
# Install the packages
! pip3 install --upgrade google-cloud-aiplatform \
                         google-cloud-bigquery\
                         google-cloud-storage\
                         bigframes\
                         pandas-gbq\
                         db-dtypes

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
# BigQuery関連の設定
PROJECT_ID = ""  # @param {type:"string"} Google CloudプロジェクトID
REGION = "us-central1"  # @param {type:"string"} 使用するリージョン
BQ_DATASET_ID = ""  # @param {type:"string"} BigQueryのデータセットID
BQ_TABLE_ID = ""  # @param {type:"string"} BigQueryのテーブルID

# Feature Storeの設定
FEATURE_ONLINE_STORE_ID = ""  # @param {type:"string"} Feature StoreのオンラインストアID
FEATURE_VIEW_ID = ""  # @param {type:"string"} Feature StoreのビューID

# スケジュール設定
# スケジュールはCRON設定に基づいて作成されます。
# CRONが空の場合、即時スケジュールジョブが開始されます。
CRON_SCHEDULE = "TZ=Asia/Tokyo 0 9 * * *"  # @param {type:"string"} スケジュール設定（東京時間で毎日午前9時）

# ベクトル検索の設定
DIMENSIONS = 1408  # @param {type:"number"} ベクトルの次元数
EMBEDDING_COLUMN = "embedding"  # @param {type:"string"} 埋め込みを保持する列名

# オプショナル設定
LEAF_NODE_EMBEDDING_COUNT = 10000  # @param {type:"number"} リーフノードの埋め込み数（オプショナル）
FILTER_COLUMNS = ["title"]  # @param {type:"string"} フィルタリングに使用する列（オプショナル）

# Feature Store のデータソースとなる BigQuery テーブルを定義
BQ_TABLE_ID_FQN = f"{BQ_DATASET_ID}.{BQ_TABLE_ID}"
DATA_SOURCE = f"bq://{PROJECT_ID}.{BQ_TABLE_ID_FQN}"

# オンラインストアのエンドポイント
API_ENDPOINT = f"{REGION}-aiplatform.googleapis.com"

In [ ]:
# Google Cloud AI Platform関連のインポート
from google.cloud import aiplatform
from google.cloud.aiplatform_v1beta1 import (
    FeatureOnlineStoreAdminServiceClient,
    FeatureOnlineStoreServiceClient
)
from google.cloud.aiplatform_v1beta1.types import (
    NearestNeighborQuery,
    feature_online_store as feature_online_store_pb2,
    feature_online_store_admin_service as feature_online_store_admin_service_pb2,
    feature_online_store_service as feature_online_store_service_pb2,
    feature_view as feature_view_pb2
)
from google.protobuf import struct_pb2

# Google Cloud BigQuery関連のインポート
from google.cloud import bigquery

# Google Cloud Storage関連のインポート
from google.cloud import storage

# その他のインポート
import bigframes.pandas as bpd
import random
import base64
import time
import typing


aiplatform.init(project=PROJECT_ID, location=REGION)

admin_client = FeatureOnlineStoreAdminServiceClient(
    client_options={"api_endpoint": API_ENDPOINT}
)

API_ENDPOINT = f"{REGION}-aiplatform.googleapis.com"

In [ ]:
# BigQuery ソースを定義
big_query_source = feature_view_pb2.FeatureView.BigQuerySource(
    uri=DATA_SOURCE, entity_id_columns=["image_path"]
)

# オンラインストア用の Config 設定
online_store_config = feature_online_store_pb2.FeatureOnlineStore(
    bigtable=feature_online_store_pb2.FeatureOnlineStore.Bigtable(
        auto_scaling=feature_online_store_pb2.FeatureOnlineStore.Bigtable.AutoScaling(
            min_node_count=1, max_node_count=3, cpu_utilization_target=50
        )
    ),
    embedding_management=feature_online_store_pb2.FeatureOnlineStore.EmbeddingManagement(
        enabled=True
    ),
)

#  オンラインストアを作成する（新規）
create_store_lro = admin_client.create_feature_online_store(
    feature_online_store_admin_service_pb2.CreateFeatureOnlineStoreRequest(
        parent=f"projects/{PROJECT_ID}/locations/{REGION}",
        feature_online_store_id=FEATURE_ONLINE_STORE_ID,
        feature_online_store=online_store_config,
    )
)


# オンラインストアが正常に作成されたかを確認
admin_client.get_feature_online_store(
    name=f"projects/{PROJECT_ID}/locations/{REGION}/featureOnlineStores/{FEATURE_ONLINE_STORE_ID}"
)

In [ ]:
# BigQuery から オンラインストアへの同期ジョブのスケジュール設定
sync_config = feature_view_pb2.FeatureView.SyncConfig(cron=CRON_SCHEDULE)

# ベクトル検索の高速化設定を定義
vector_search_config = feature_view_pb2.FeatureView.VectorSearchConfig(
    embedding_column=EMBEDDING_COLUMN,
    filter_columns=FILTER_COLUMNS,
    # crowding_column=CROWDING_COLUMN,
    embedding_dimension=DIMENSIONS,
    tree_ah_config=feature_view_pb2.FeatureView.VectorSearchConfig.TreeAHConfig(),
)

# 特徴量ビューを作成
create_view_lro = admin_client.create_feature_view(
    feature_online_store_admin_service_pb2.CreateFeatureViewRequest(
        parent=f"projects/{PROJECT_ID}/locations/{REGION}/featureOnlineStores/{FEATURE_ONLINE_STORE_ID}",
        feature_view_id=FEATURE_VIEW_ID,
        feature_view=feature_view_pb2.FeatureView(
            big_query_source=big_query_source,
            sync_config=sync_config,
            vector_search_config=vector_search_config,
        ),
    )
)

# 特徴量ビューが作成されたか確認
admin_client.get_feature_view(
    name=f"projects/{PROJECT_ID}/locations/{REGION}/featureOnlineStores/{FEATURE_ONLINE_STORE_ID}/featureViews/{FEATURE_VIEW_ID}"
)

# その他の特徴量ビューについて確認
admin_client.list_feature_views(
    parent=f"projects/{PROJECT_ID}/locations/{REGION}/featureOnlineStores/{FEATURE_ONLINE_STORE_ID}"
)

In [ ]:
# 手動での同期ジョブの実行
sync_response = admin_client.sync_feature_view(
    feature_view=f"projects/{PROJECT_ID}/locations/{REGION}/featureOnlineStores/{FEATURE_ONLINE_STORE_ID}/featureViews/{FEATURE_VIEW_ID}"
)

In [ ]:
import time

# 同期ジョブが完了したかのポーリング
while True:
    feature_view_sync = admin_client.get_feature_view_sync(
        name=sync_response.feature_view_sync
    )
    if feature_view_sync.run_time.end_time.seconds > 0:
        status = "Succeed" if feature_view_sync.final_status.code == 0 else "Failed"
        print(f"Sync {status} for {feature_view_sync.name}.")
        # wait a little more for the job to properly shutdown
        time.sleep(30)
        break
    else:
        print("Sync ongoing, waiting for 30 seconds.")
    time.sleep(30)

In [ ]:
# オンラインストアのインスタンスからパブリックエンドポイントを取得
featore_online_store_instance = admin_client.get_feature_online_store(
    name=f"projects/{PROJECT_ID}/locations/{REGION}/featureOnlineStores/{FEATURE_ONLINE_STORE_ID}"
)
PUBLIC_ENDPOINT = (
    featore_online_store_instance.dedicated_serving_endpoint.public_endpoint_domain_name
)

# 近傍検索の時に以下のURLを指定
print(f"PUBLIC_ENDPOINT for online serving: {PUBLIC_ENDPOINT}")